In [1]:
import torch
import torch.nn as nn
from models import GridMLMMelHarm
from GridMLM_tokenizers import CSGridMLMTokenizer
from data_utils import CSGridMLMDataset, CSGridMLM_collate_fn
from torch.utils.data import DataLoader
from train_utils import apply_masking
from generate_utils import generate_files_with_base2, generate_files_with_random, load_model

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
val_dir = '/media/maindisk/maximos/data/hooktheory_all12_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies/'
tokenizer = CSGridMLMTokenizer(fixed_length=256)
# val_dataset = CSGridMLMDataset(val_dir, tokenizer, 512)

In [3]:
curriculum_type = 'random'
# curriculum_type = 'base2'
subfolder = 'all12'
device_name = 'cuda:2'
model = load_model(
    curriculum_type=curriculum_type,
    subfolder=subfolder,
    device_name='cuda:2',
    tokenizer=tokenizer,
    pianoroll_dim=100
)

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

26203106


In [5]:
import os
data_files = []
for dirpath, _, filenames in os.walk(val_dir):
    for file in filenames:
        if file.endswith('.xml') or file.endswith('.mxl') or file.endswith('.musicxml'):
            full_path = os.path.join(dirpath, file)
            data_files.append(full_path)
print(len(data_files))

10486


In [6]:
val_idx = 0

# input_f = data_files[val_idx]
input_f = 'example_inputs/test_midi_constraints.mid'

mxl_folder = 'examples_musicXML/' + subfolder + '/' + curriculum_type + '/' + '/'
midi_folder = 'examples_MIDI/' + subfolder + '/' + curriculum_type + '/' + '/'
os.makedirs(mxl_folder, exist_ok=True)
os.makedirs(midi_folder, exist_ok=True)

gen_harm, real_harm, gen_score, real_score = generate_files_with_random(
    model=model,
    tokenizer=tokenizer,
    input_f=input_f,
    mxl_folder=mxl_folder,
    midi_folder=midi_folder,
    name_suffix='constr_test',
    use_constraints=True
)

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/base.py:559: UserWarning: Setting an ID that could be mistaken for a memory location is discouraged: got 140659322990144
  warnings.warn(msg)
libpng warning: iCCP: known incorrect sRGB profile
Error at line 30 col 47: no instrument found for part 'P01fb5dcf9a3aa2300fde918868be60dd'
Error at line 637 col 47: no instrument found for part 'P8ea10213c85f16e7b5aa79d082be15b5'
convert <examples_musicXML/all12/random//gen_constr_test.mxl>...
	to <examples_MIDI/all12/random//gen_constr_test.mid>
... success!
libpng warning: iCCP: known incorrect sRGB profile
Error at line 30 col 47: no instrument found for part 'P86c11a5ec1ba58a41df064523bd17591'
Error at line 637 col 47: no instrument found for part 'Peea6714be31eae2ffc9f67207ceee49c'
convert <examples_musicXML/all12/random//real_constr_test.mxl>...
	to <examples_MIDI/all12/random//real_constr_test.mid>
... success!


In [7]:
print(gen_harm)
print(tokenizer.ids_to_tokens[6])

['<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', 'D:maj', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', 'C:maj', 'C:maj', 'C:maj', 'C:maj', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<nc>', '<

In [8]:
real_score.show('t')

{0.0} <music21.metadata.Metadata object at 0x7fedcd77b740>
{0.0} <music21.stream.Part 0x7fedcd8e5b20>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.clef.BassClef>
        {0.0} <music21.tempo.MetronomeMark allegro moderato Quarter=128>
        {0.0} <music21.key.Key of G major>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note G>
        {0.25} <music21.note.Rest eighth>
        {0.75} <music21.note.Note G>
        {1.25} <music21.note.Rest 16th>
        {1.5} <music21.note.Note G>
        {2.0} <music21.note.Note B>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note A>
        {3.5} <music21.note.Note G>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note G>
        {0.25} <music21.note.Rest eighth>
        {0.75} <music21.note.Note G>
        {1.25} <music21.note.Rest 16th>
        {1.5} <music21.note.Note G>
        {2.0} <music21.note.Note B>
        {2.5} <music21.note.Note D>
  